# Data

### Send requests to OpenAI API

In [ ]:
import naas_python
import pandas as pd
import requests
import json
import time
import naas_python

def create_naas_chat_completion(
    access_token=None,
    model_id="63b149d9-f8fe-4b33-8132-75e7861c347d",
    prompt="",
    message="",
):
    # Requests
    url = f"https://api.naas.ai/model/{model_id}/completion"
    payload = json.dumps(
        {
            "id": model_id,
            "payload": json.dumps(
                {
                    "messages": 
                    [
                        {
                            "role": "system",
                            "content": prompt
                        },
                        {
                            "role": "user",
                            "content": message
                        }
                    ]
                }
            )
        }
    )
    headers = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {access_token}'
    }
    result = None
    retry = 0
    while True:
        try:
            response = requests.post(url, headers=headers, data=payload)
            result = response.json()['completion']['completions'][0]
            break
        except Exception as e:
            print(e)
        retry += 1
        time.sleep(10)
        if retry >= 3:
            break
    return result

def get_ia_analysis(
    df=pd.DataFrame(),
    prompt_data="",
    api_key=None
):
    # Init
    result = ""
    
    # Check if dataframe empty
    if len(df) == 0:
        return result
        
    # Params
    prompt = f"""
    Tu es un analyste comptable et financier.
    Tu dois réaliser une analyse en 3 phrases avec une limite de 350 caractères sur le jeu de donnée que tu vas recevoir.
    Les chiffres doivent être écrit au format suivant: 26 052 €.
    {prompt_data}
    """
    
    # Set api key
    if api_key is None:
        api_key = naas_python.utils.domains_base.authorization.NaasSpaceAuthenticatorAdapter().jwt_token()

    # Create chat completion
    result = create_naas_chat_completion(
        access_token=api_key,
        prompt=prompt,
        message=df.to_string(),
    )

    print("🤖 IA:", result)
    return result